# 1. Data science workflow

![workflow](imgs/data_science_workflow.png)

## We are going to focus on the "Wrangle->Clean->Explore" part.  
- "Data wrangling:" cleaning and unifying messy/complex data sets for easy access and analysis
- The majority of a data scientists time is spent on data cleaning/wrangling!
- "Garbage in/garbage out:"  Incorrect or poor quality input will always produce faulty output.  Examples:
    - Some rows in a table are duplicated
    - Some data entries are outside the realistic range (e.g. negative values for age)
    - Data combined from two sources uses two different scales (e.g. celsius vs. Farenheit)

![pic](imgs/forbes_data_sci.jpg)
[Source](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#96b939c6f637)

# 2.  Setup - installing pandas

To use this pandas tutorial, we'll need to make sure we have access to:
- python3
- pandas library
- Jupyter notebooks

You have a few options:
1. Cloud environment (start here if you're not command line savvy and/or haven't used jupyter notebooks before): 
    - Use the JupyterLab binder button on the github repo. All of the dependencies are included in the environment.yml file used to build the binder container; you do not need to install anything on your computer.
2. Local environment, option 1: 
    - Clone the repo (`git clone https://github.com/marskar/snakemake.git`)
    - Use conda (or your favorite package manager) to install the dependencies locally (`conda install -c bioconda <tool=version>`)
3. Local environment, option 2:
    - Clone the repo (`git clone https://github.com/marskar/snakemake.git`)
    - Use the environment file to build a conda environment locally (`conda env create -f environment.yml`)
    - Be sure to start up jupyter notebook from within this environment!

# 3.  About pandas

## What is pandas?

- Package for cleaning, transforming, and analyzing your data
- Built on top of NumPy, with additional functionality built in to deal with messier data
    - Attach row and column labels
    - Work with missing data
    - Handle heterogeneous data types
    
## Why use pandas?

- Have you ever used an Excel spreadsheet?  What were some of the ways you gleaned insights from your data?

- Using pandas to clean and explore your data can be followed by application of machine learning models, or it can be a standalone analysis, depending on your needs

The two central components in pandas are the `series` and the `dataframe`.  A `series` is a column of data, and a `dataframe` is a table made up of a group of `series`.  Visually:

__Series:__

|Age|
|---|
|3|
|7|
|18|
|54|
|12|

__Dataframe:__

|Age|Sex|Height|
|---|---|---|
|3|M|36|
|7|F|45|
|18|F|60|
|54|M|66|
|12|M|58|



First, import the numpy and pandas libraries.

In [ ]:
import numpy as np
import pandas as pd

# 4.  Building series and dataframes from scratch

To get started, let's build a couple series from numpy arrays.

In [ ]:
age_array = np.array([3,7,18,54,12])
age_array

In [ ]:
age_series = pd.Series(age_array)
age_series

In [ ]:
sex_array = np.array(['M','F','F','M','M'])
sex_series = pd.Series(sex_array)
sex_series

Now, let's build a dataframe from scratch.  There are many ways to do this.  Let's first build a dataframe from a multidimensional numpy array.

In [ ]:
df1 = pd.DataFrame(np.array([[3,7,18,54,12], ['M','F','F','M','M']]).T)
df1

Alternatively, we could build a dataframe from scratch using a dict.  A dict is composed of __key:value__ pairs (in this case, the keys are age and sex; each has a single value which is a python list).  

In [ ]:
data = {
    'age': [3,7,18,54,12], 
    'sex': ['M','F','F','M','M']
}

In [ ]:
df2 = pd.DataFrame(data)
df2

# 5.  Importing data

So, that's pretty cool, but it's unlikely that you'll want to manually type in your whole dataset as an array or a dict.  How can we read external data into a dataframe?  Let's say we have a tab-separated text file with data from an esophageal cancer project.

In [ ]:
df_orig = pd.read_csv('data/esophageal_subjects.txt', sep='\t')
df_orig

If you're running this notebook in Google Colab, use the following cell to pull in the data (this will also work in Binder).

In [ ]:
url = 'https://raw.githubusercontent.com/bballew/pandas_tutorial/master/data/esophageal_subjects.txt'
df_orig = pd.read_csv(url, sep='\t')
df_orig

# 6.  Viewing the dataframe

In [ ]:
df_orig.head()

__Exercise: First, put the cursor in the following cell, then press `shift + TAB`.  If that does nothing, prepend the cell with a '?' and run it.  Depending on your environment, one of these two options will pull up documentation.  Then, edit the cell to view the first 10 lines of the dataframe__

In [ ]:
df_orig.head()

In [ ]:
df_orig.tail()

In [ ]:
df_orig.shape

In [ ]:
df_orig.info()

Note the data types reported above.  For example, 'C3_EDUC' looks like a column of numbers, but the dtype is reported as "object", which is a string.  This is a clue that something might be awry in this column (for example, there may be a missing value that is noted with a string like 'na').

Count non-null values by column.

In [ ]:
df_orig.count()

Count distinct values by column.  Check here that columns have the expected number of unique values, e.g. two values for sex.

In [ ]:
df_orig.nunique()

# 7.  Clean up the data

## 7.1.  Remove duplicate values

You may want to check your data for duplicate entries.  Using `drop_duplicates()`, we can remove all duplicate rows, or keep only the first or last row of a set of duplicates.  Here, we remove a row in which all values are duplicates of another row.

In [ ]:
df_clean = df_orig.copy()
df_clean.head()

In [ ]:
df_clean.sort_values("ID", inplace = True)
df_clean.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
df_clean.shape

We can also remove duplicates based on specific values, like "Dx".  In this example, we keep only the last row of each duplicate diagnosis.

In [ ]:
df_Dx = df_clean.sort_values('Dx')
df_Dx.drop_duplicates(subset = 'Dx', keep = 'last', inplace = True)
df_Dx.shape

In [ ]:
df_Dx

__Exercise: Copy the dataframe df_orig, sort the copy by sex, drop all but the first duplicates, and determine the shape__

## 7.2.  Modify column headings and row labels

Let's look at the column headings in our data.

In [ ]:
df_clean.columns

The columns headings on our imported data are difficult to interpret.  Let's change them to something that makes more sense.

In [ ]:
df_clean.rename(columns={'C3_EDUC':'Education_level',
                   'C16_HFUEL':'Heating_source',
                   'D1_EVCIG':'Smoke_6mos',
                   'D2_START':'Age_start_smoking',
                   'D4_STOP':'Age_stop_smoking',
                   'D5_NUMCIG':'Cigarettes_per_day',
                   'D8_LIQ':'Liquor_freq',
                   'E1_UGICAN':'Family_hx_UGIcancer',
                   'G1_TEMP':'Temp_of_hot_beverages',
                   'G2_SFREQ':'Frequency_scalding_food',
                   'H1_HEIGHT':'Height_cm',
                   'H2_WEIGHT':'Weight_kg'},
         inplace=True)

In [ ]:
df_clean.head()

We could also change the column headings using `.columns`.  Let's say we wanted a dataframe with only lowercase column headings.

In [ ]:
df_clean.columns = ['id', 'dx', 'age', 'sex', 'education_level', 'heating_source',
       'smoke_6mos', 'age_start_smoking', 'age_stop_smoking',
       'cigarettes_per_day', 'liquor_freq', 'family_hx_ugicancer',
       'temp_of_hot_beverages', 'frequency_scalding_food', 'height_cm',
       'weight_kg', 'ps_id', 'wgs']

In [ ]:
df_clean.head()

Our row labels right now are just numbered, i.e. 0,1,2...  Let's change the row labels to the patient IDs.

In [ ]:
df_clean.set_index('id', inplace=True)

In [ ]:
df_clean.head()

## 7.3.  Re-coding data

This data has many encoded values, e.g. integers that represent various levels of education, heating fuel types, etc.  Let's replace the encoded data with the real data.

In [ ]:
df_clean.columns

In [ ]:
df_clean['education_level'].replace(['1','2','3','4','5','6','7'],['None','1-5yrs','Primary','Middle','High','College','Other'], inplace=True)
df_clean['heating_source'].replace([1,2,3,4,5,6,7,8],['Coal','Coke','Coal_product','Firewood','Diesel','Kerosene','Gas','Other'], inplace=True)
df_clean['smoke_6mos'].replace([1,2],['Yes','No'], inplace=True)
df_clean['liquor_freq'].replace([1,2,3,4,5],['Daily','Weekly','Monthly','Seasonally','Rarely-never'], inplace = True)
df_clean['family_hx_ugicancer'].replace(['1','2'],['Yes','No'], inplace = True)
df_clean['temp_of_hot_beverages'].replace([1,2,3,4,9],['Cool','Warm','Hot','Scalding','DK'], inplace=True)
df_clean['frequency_scalding_food'].replace([1,2,3,4,9],['Seldom-never','Occasionally','Weekly','Daily','DK'], inplace=True)

What's the deal with `[1,2,3,]` vs. `['1','2','3']` above?  Recall the result of `.info()`.  Columns that contain only integers like 1,2,3 are of an integer dtype.  However, if there are mixed types within the column (e.g. a missing value like '.' or 'Nan'), then the column will be an object type, which are strings.

In [ ]:
df_clean.head()

__Exercise: Copy df_clean and replace 'M' and 'F' with 'male' and 'female', then show the first 5 rows__

## 7.4.  Missing values

Datasets often have missing values.  These can be annotated in a variety of ways - 'None', 'NaN', 'na', '.', a blank space...  These must first be identified.  Then, we can either drop the relevant rows/columns, or replace the missing values with something (often the mean or median of the column).  Some pandas techniques will handle missing values by simply omitting them from the calculation, so depending on what you're doing, you may be able to tolerate some missingness.

In [ ]:
df_clean.isnull()

In [ ]:
df_clean.isnull().sum()

__Exercise: Use `shift + TAB` or prepend with a `?` to look at the docustring for `.sum()`.  How would we get the sum of null values in each row, instead of each column?__

Let's take a closer look.  What type of "missing" values does `isnull()` detect?

In [ ]:
df_clean['education_level']

In [ ]:
df_clean['education_level'].isnull().sum()

Hmm...`isnull()` doesn't recognize all of our missing values.  We'll need to address this by normalizing all the missing values that we can find.

In [ ]:
df_clean.replace(['None','.','--','na'],np.nan, inplace=True)

In [ ]:
df_clean['education_level']

In [ ]:
df_clean.isnull().sum()

Great, now we can detect all our missing values.  Let's deal with missing data for the smoking-related columns.  First, let's make sure that the missing data in 'age_start_smoking' and 'age_stop_smoking' isn't due to that subject being a non-smoker.

In [ ]:
df_clean[df_clean[['smoke_6mos','age_start_smoking','age_stop_smoking','cigarettes_per_day']].isnull().any(axis=1)]

Looks like most of the missing data in these columns is because the individual was not a smoker.  Let's replace the one valid missing data field with the column mean.

In [ ]:
stop_mean = round(df_clean['age_stop_smoking'].mean())
stop_mean

In [ ]:
df_clean.at['PS002','age_stop_smoking']=stop_mean

In [ ]:
df_clean.loc[['PS002']]

Now, let's impute the missing values for age by replacing with the column mean.

In [ ]:
age_mean = round(df_clean['age'].mean())
age_mean

In [ ]:
df_clean['age'].fillna(age_mean, inplace=True)

In [ ]:
df_clean.isnull().sum()

We could also opt to remove some data, either rows or columns, if there are missing values.

In [ ]:
df_drop = df_clean.dropna() # drop rows
df_drop.shape

In [ ]:
df_drop

__Exercise: We used the mean of 'age' to fill in missing values in df_clean.  What if we wanted to use the median?__

## 7.5.  Check for biologically unrealistic values

Sometimes, a value isn't technically missing, but it's clearly outside realistic expectations.  Let's check for examples of this.

In [ ]:
df_clean['age'].describe()

In [ ]:
df_clean['height_cm'].describe()

In [ ]:
df_clean['weight_kg'].describe()

Uh oh!  It seems unlikely that someone would be 773 cm tall, or weigh 5 kg.  Look at a few quick plots to make sure there aren't other aberrant values like this.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_clean.boxplot()

In [ ]:
df_clean['weight_kg'].hist()

In [ ]:
df_clean['height_cm'].hist()

Looks like those two are the only obviously incorrect values.  Let's replace them with the mean of the columns.

In [ ]:
height_mean = round(df_clean['height_cm'].mean())
height_mean

In [ ]:
df_clean['height_cm'].replace(773, height_mean, inplace=True)

In [ ]:
weight_mean = round(df_clean['weight_kg'].mean())
weight_mean

In [ ]:
df_clean['weight_kg'].replace(5, weight_mean, inplace=True)

Now let's re-check the distributions, to make sure everything looks plausible.

In [ ]:
df_clean.boxplot()

__Exercise: Take a closer look at the 'age' data by plotting a histogram.__

## 7.6  Data types

Now that we've done a lot of work cleaning our data, let's make sure each column contains the expected data type.

In [ ]:
df_clean.info()

The column 'cigarettes_per_day' is listed as object, which is a string type.  Why?

In [ ]:
df_clean['cigarettes_per_day'].value_counts()

In [ ]:
df_clean['cigarettes_per_day']

NaN is a float-representation of missing data; the other entries in this column are integers.  Let's change them all to floats.

In [ ]:
df_clean['cigarettes_per_day'] = df_clean['cigarettes_per_day'].astype('float64')

In [ ]:
df_clean.info()

# 8.  Slicing and extracting data

You have seen some of these techniques in action already.  You can subset a dataframe in several ways, and the subsetted data can be either a dataframe or a series.  

First, let's subset by column name.  Notice that this results in a series.

In [ ]:
df_clean['age']

In [ ]:
type(df_clean['age'])

You can subset by column name and get a dataframe by passing a list.

In [ ]:
df_clean[['age']]

In [ ]:
type(df_clean[['age']])

You can easily subset a dataframe by multiple columns, by adding column names to the list.

In [ ]:
df_clean[['age','weight_kg']]

You can also extract data from rows in the dataframe by index or by label, using `.loc` and `.iloc`. 

In [ ]:
df_clean.loc['PS002']

In [ ]:
df_clean.iloc[0]

As with columns, the above methods give you a series.  You can get a dataframe by providing a list.

In [ ]:
df_clean.loc[['PS002']]

__Exercise: Look at the column headers in df_clean by using `.columns`, then select one to extract from the dataframe.  How do you extract the data as a series?  As a dataframe?__

# 9.  Filter and group data

You can subset your dataframe using conditional selections.

First, let's see what happens if we make a conditional selection.

In [ ]:
df_clean['heating_source'] == 'Coal'

Now, let's apply that selection to the dataframe to retrieve the relevant rows.

In [ ]:
df_clean[df_clean['heating_source'] == 'Coal']

__Exercise: Retrieve all rows from df_clean where the heating source is anything other than 'Coal'.__

You can also select continuous variables in a similar way.

In [ ]:
df_clean[df_clean['age'] > 65]

You can apply multiple criteria to your selection using or (`|`) or and (`&`).  Note the use of parentheses to group conditional statements.

In [ ]:
df_clean[(df_clean['temp_of_hot_beverages'] == 'Scalding') | (df_clean['temp_of_hot_beverages'] == 'Hot')]

Using `.isin()` is more concise than the above command, but does the same thing.

In [ ]:
df_clean[df_clean['temp_of_hot_beverages'].isin(['Scalding', 'Hot'])]

This is an `&` example.

In [ ]:
df_clean[(df_clean['temp_of_hot_beverages'] == 'Scalding') & (df_clean['frequency_scalding_food'] == 'Daily')]

You can group your data using `groupby()`.  For example, let's say we want to calculate the mean age for each category of "temp_of_hot_beverages."

In [ ]:
df_clean.groupby(['temp_of_hot_beverages'])['age'].mean()

__Exercise: Now that we know `.groupby()`, how could we have used it to check that non-smoking status ('smoke_6mos') correlates correctly with missing values for 'age_start_smoking', 'age_stop_smoking', and 'cigarettes_per_day?'__

# 10.  Apply filters, groupby, etc. to generate insightful plots

First, let's plot what we just calculated, the mean age for each category of "temp_of_hot_beverages."

In [ ]:
df_clean.groupby(['temp_of_hot_beverages'])['age'].mean().plot.bar()

How much do our subjects drink?  Let's use `.value_counts()` to determine how many instances there are of each "liquor_freq" category, and plot the result.

In [ ]:
df_clean['liquor_freq'].value_counts()

In [ ]:
df_clean['liquor_freq'].value_counts().plot.bar()

Let's combine filtering and plotting to look at the distribution of smokers' start and stop ages, amongst subjects who drink scalding beverages.

In [ ]:
df_clean[df_clean['temp_of_hot_beverages'] == 'Scalding'].boxplot(column=['age_start_smoking','age_stop_smoking'])

Let's use `groupby()` again, this time to look at the means for all continous variables, grouped by education level.

In [ ]:
df_clean.groupby(['education_level']).mean().plot.bar(figsize=[10,5])

That's a rather busy graph - let's redraw it, just focusing on smoking start/stop ages.  Do you think there's likely to be a difference in start/stop age across different levels of education?

In [ ]:
df_clean.groupby(['education_level'])['age_start_smoking','age_stop_smoking'].mean().plot.bar(figsize=[10,5])

We can create scatter plots to look at the interaction between multiple features.  Here, let's look at height and weight.

In [ ]:
df_clean.plot.scatter('height_cm','weight_kg')

We can use `corr()` to look for correlations within our data.  Without any additional parameters, this will compare all continuous variables in your dataframe.

In [ ]:
df_clean.corr()

There are lots of ways to create a heatmap from this data.  Here's just one simple one, that doesn't rely on any additional libraries.

In [ ]:
df_clean.corr().style.background_gradient(cmap='autumn')

__Exercise: Generate a scatter plot of any two continuous variables.__

__Exercise: Plot the distribution of all continuous variables for subjects below 65 and above 65.__ 

# 11.  Conclusions

Make sure to download the notebook if you've made changes and want to save them!  If you re-open the binder link, it will be reset to the way you originally found it.

Already use Excel, and want to start using pandas on your own?  Try this challenge: open your data in Excel and read it into pandas.  For every action you perform in Excel, do the equivalent thing in pandas.